In [1]:
import os

import geopandas as gpd
import osmnx as ox
from geopandas import GeoDataFrame, GeoSeries
from osmnx import io
import glob

project_crs = 'epsg:3857'
from sklearn.cluster import DBSCAN
from shapely.geometry import Polygon, Point, LineString, MultiPolygon, MultiPoint
import math
import warnings
import pandas as pd
import shutil
from tqdm import tqdm
import time
warnings.filterwarnings(action='ignore')
from momepy import remove_false_nodes,extend_lines
pjr_loc = os.path.dirname(os.getcwd())



<span style="color: Green;font-size: 30px">Module 1:Preliminary work</span>
<ul> <li>Download data from OpenStreetMap, project it, and convert it to a GeoDataFrame. OSMnx automatically resolves topology errors and retrieves only the street-related polylines.</li>
 <li>Identify roundabout elements, if any exist, and store them in a separate DataFrame.</li>
  <li>Remove additional irrelevant line objects based on values of the OSM 'tunnel' and 'highway' keys.</li>
   <li>Eliminate polylines that lack a name and calculate angles ranging from 0 to 180 degrees based on the bearing field.</li>
   </ul>


<span style="color: Red;font-size: 30px">Module 2 -3:Detect  parallel streets segments and merge them </span>
<ul> <li>For each group of streets with the same name search for parallel segments</li>
 <li> Use DBSCAN to classify streets based on their angle, and group each class. Outliers could not consider parallel with any street, thus removed</li>
  <li>The parallel test is on street segments that  have the same name and belong to the same angle group.
    <ul><li>Eliminate polylines that lack a name and calculate angles ranging from 0 to 180 degrees based on the bearing field.</li></ul>
    </li>

   </ul>

In [2]:
# In this example, the data is extracted from OSM by specifying a location's name, but you can also download data using a specified polygon. The code is designed to handle multiple polygons or location names seamlessly.

# Download data from OpenStreetMap, project it, and convert it to a GeoDataFrame. OSMnx automatically resolves topology errors and retrieves only the street-related polylines.

place = 'Turin'
print(place)
data_folder  = f'places/{place.replace(",","_").replace(" ","_")}_test'

Turin


In [ ]:
graph = ox.graph_from_place(place, network_type='all')
graph = ox.bearing.add_edge_bearings(graph, precision=1)
graph_pro = ox.projection.project_graph(graph, to_crs=project_crs)
io.save_graph_geopackage(graph_pro, filepath=f'{data_folder}/osm_data.gpkg', encoding='utf-8', directed=False)


In [9]:
# find and store roundabout
my_gdf = gpd.read_file(f'{data_folder}/osm_data.gpkg',layer = 'edges')# Identify roundabout elements, if any exist, and store them in a separate DataFrame.
is_junction= True if 'junction' in my_gdf.columns else False
if is_junction:
    round_about = my_gdf[my_gdf['junction'].isin(['roundabout', 'circular'])]
    my_gdf= my_gdf[~((my_gdf['junction'] == 'roundabout') | (my_gdf['junction'] == 'circular'))]

In [ ]:

# Remove additional irrelevant line objects based on values of the OSM 'tunnel' and 'highway' keys.
if 'tunnel' in my_gdf.columns:
    my_gdf = my_gdf[~((my_gdf['tunnel'] == 'building_passage') | (my_gdf['tunnel'] == 'yes'))]
to_remove = my_gdf[~((my_gdf['highway'] == 'motorway') | (my_gdf['highway'] == 'trunk')| (my_gdf['highway'] == 'motorway_link')| (my_gdf['highway'] == 'motorway_link')| (my_gdf['highway'] == 'trunk_link'))]


# Eliminate polylines that lack a name and calculate angles ranging from 0 to 180 degrees based on the bearing field.
df_pro = to_remove.to_crs(project_crs).dropna(subset=['name'])
df_pro = df_pro[df_pro['name']!='']
df_pro['angle'] = df_pro['bearing'].apply(lambda x: x if x < 180 else x - 180)
df_pro['length'] = df_pro.length

In [8]:

# Functions and classes to be utilized - Module 2
def length_of_parallel(my_s_join: GeoDataFrame, the_buffer: GeoSeries, geo_field: str) -> int:
    my_s_join['geometry'] = my_s_join[geo_field]
    new_data_0 = my_s_join.sjoin(GeoDataFrame(geometry=the_buffer, crs=project_crs), how='inner').reset_index()
    if len(new_data_0) == 0:
        return 0
    return len(new_data_0[new_data_0['index'] != new_data_0['index_right']])
def check_parallelism(to_translate: GeoDataFrame) -> bool:
    my_buffer = to_translate['geometry'].buffer(cap_style=2, distance=30, join_style=3)
    to_translate['geometry_right'] = to_translate['geometry'].apply(lambda x: x.parallel_offset(35, 'right'))
    to_translate['geometry_left'] = to_translate['geometry'].apply(lambda x: x.parallel_offset(35, 'left'))
    if length_of_parallel(to_translate, my_buffer, 'geometry_right') > 10 or length_of_parallel(to_translate, my_buffer, 'geometry_left') > 10:
        return True
    else:
        return False

# Functions and classes to be utilized - Module 3
def update_list(line_local):
    """
    add the first start/end point into the list
    :param line_local:
    :return:
    """
    list_pnts_of_line_group.extend([Point(line_local.coords[0]), Point(line_local.coords[-1])])
def create_center_line(one_poly):
    """
    This method calculate new line between the farthest points of the simplified polygon
    :param one_poly:
    :return:
    """

    pnt_list = one_poly.exterior.coords[:-1]
    list_shp = [Point(item) for item in pnt_list]
    dis, dis_2, dis_3 = 0, 0, 0
    third_dis = (-1, -1)
    # The new line will be determined by the third-farthest points
    for k, point in enumerate(list_shp):
        j = k
        for point2 in list_shp[k + 1:]:
            j += 1
            temp_dis = point.distance(point2)
            if temp_dis > dis:
                dis = point.distance(point2)
            elif temp_dis > dis_2:
                dis_2 = point.distance(point2)
            elif temp_dis > dis_3:
                third_dis = (k, j)
                dis_3 = point.distance(point2)
    max_dist['name'].extend([id_pol + 2, id_pol + 2])
    max_dist['geometry'].extend([list_shp[third_dis[0]], list_shp[third_dis[1]]])
def add_more_pnts_to_new_lines(pnt_f_loc: Point, pnt_l_loc: Point, line_pnts: list) -> list:
    """
    This method checks if more points should be added to the new lines by checking along the new line if the distance to the old network roads are more than 10 meters
    :return:
    """
    # Calculate distance and azimuth between the first and last point
    dist = pnt_f_loc.distance(pnt_l_loc)
    x_0 = pnt_f_loc.coords[0][0]
    y_0 = pnt_f_loc.coords[0][1]
    bearing = math.atan2(pnt_l_loc.coords[0][0] - x_0, pnt_l_loc.coords[0][1] - y_0)
    bearing = bearing + 2 * math.pi if bearing < 0 else bearing
    # Calculate the number of  checks going to carry out
    loops = int(dist / 100)

    # Calculate  the first point over the line
    for dis_on_line in range(1, loops):
        x_new = x_0 + 100 * dis_on_line * math.sin(bearing)
        y_new = y_0 + 100 * dis_on_line * math.cos(bearing)
        # S_joins to all the network lines (same name and group)
        # if the distance is less than 10 meters continue, else: find the projection point and add it to the correct location and run the function agein
        one_pnt_df = GeoDataFrame(geometry=[Point(x_new, y_new)], crs=project_crs)
        s_join_loc = one_pnt_df.sjoin_nearest(data, distance_col='dis').iloc[0]
        if s_join_loc['dis'] > 10:
            pnt_med = s_join_loc['geometry']
            line = data.loc[s_join_loc['index_right']]['geometry']
            line_pnts.append(line.interpolate(line.project(pnt_med)))
            line_pnts = add_more_pnts_to_new_lines(pnt_med, pnt_l_loc, line_pnts)
            return line_pnts
    return line_pnts
def update_df_with_center_line(new_line,is_simplified):
    """
    update our dictionary with new lines
    :param is_simplified:
    :param new_line:
    :return:
    """
    dic_final['name'].append(name)
    # dic_final['geometry'].append(LineString(coordinates=(pnt_list[max_dis[0]], pnt_list[max_dis[1]])))
    dic_final['geometry'].append(new_line)
    dic_final['highway'].append(data.iloc[0]['highway'])
    dic_final['bearing'].append(data['angle'].mean())
    dic_final['group'].append(data.iloc[0]['highway'])
    dic_final['is_simplified'].append(is_simplified)
# group the street segments by street name
my_groupby = df_pro.groupby('name')
dic_final = {'name': [], 'geometry': [], 'highway': [], 'bearing': [], 'group': [],'is_simplified':[]}

for_time = len(my_groupby)

In [9]:
is_test = True
with tqdm(total=for_time) as pbar: #  It is used in order to visualise the progress by progress bar
    for i, street in enumerate(my_groupby):
        if is_test:
            name = 'Corso Giovanni Agnelli'
            res = my_groupby.get_group(name)
        else:
            res = street[1] # it holds all the streets
            name = street[0] # It holds the streets name
                # For each group of streets with the same name search for parallel segments
        pbar.update(1) # for the progress bar
        # Remove segments without angle. If less than two segments being left move to the next group.
        res = res.dropna(subset=['angle'], axis=0)
        if len(res) < 2:
            data = res
            _  = res['geometry'].apply(lambda x:update_df_with_center_line(x,0))
            continue
        # Use DBSCAN to classify streets based on their angle, and group each class. Outliers could not consider parallel with any street, thus removed
        res['group'] = DBSCAN(eps=10, min_samples=2).fit(res['angle'].to_numpy().reshape(-1, 1)).labels_
        # if all is -1, don't touch the element
        if (res['group']== -1).all():
            data = res
            _  = res['geometry'].apply(lambda x:update_df_with_center_line(x,0))
            continue
        # cur_group = res[(res['group'] > -1) | (res.length>20)].groupby('group') # Remove short segments with -1 classification values
        # The parallel test is on street segments that  have the same name and belong to the same angle group.
        for group in res.groupby('group'):
            data = group[1]
            if group[0] ==-1: # No need to check if is parallel
                _  = data['geometry'].apply(lambda x:update_df_with_center_line(x,0))
                continue
            if check_parallelism(data.copy()):
                # if among of lines with same angles some are parallel:
                # new points DataFrame of start/end line of each group
                list_pnts_of_line_group = []
                data['geometry'].apply(update_list)
                df_pnts = GeoDataFrame(geometry=list_pnts_of_line_group, crs=project_crs).drop_duplicates()

                # unify lines to one polygon
                buffers = data.buffer(cap_style=3, distance=30, join_style=3)
                one_buffer = buffers.unary_union

                max_dist = {'name': [], 'geometry': []}
                # simplify polygon with simplify function. If one_buffer is multipolygon object simplify each one them separately
                if isinstance(one_buffer, MultiPolygon):
                    for id_pol, polygon in enumerate(one_buffer):
                        create_center_line(polygon)
                else:
                    id_pol = -1
                    create_center_line(one_buffer)
                max_df = GeoDataFrame(max_dist, crs=project_crs)
                max_df.to_file(f'{data_folder}/max_df.shp')
                # find for each points the closet point from the oribinal data. the closet points will create the new line
                s_join = max_df.sjoin_nearest(df_pnts).groupby('name')
                for geo in s_join:
                    same_name = geo[1]
                    if same_name.iloc[0]['index_right'] == same_name.iloc[1]['index_right']:
                        continue
                    in_0 = same_name.iloc[0]['index_right']
                    in_1 = same_name.iloc[1]['index_right']
                    # These points will be served to be initial reference in order to find more points
                    pnt_f = df_pnts.loc[in_0]['geometry']
                    pnt_l = df_pnts.loc[in_1]['geometry']
                    lines_pnt_geo = add_more_pnts_to_new_lines(pnt_f, pnt_l, [pnt_f])
                    lines_pnt_geo.append(pnt_l)
                    # Update dic_final
                    update_df_with_center_line(LineString(lines_pnt_geo),1)

            else:
                _  = data['geometry'].apply(lambda x:update_df_with_center_line(x,0))
        if is_test:
            break




  0%|          | 1/2493 [00:00<19:59,  2.08it/s]


TypeError: 'float' object is not subscriptable

In [ ]:
print('create new files')
# remove short lines
final_cols = ['name', 'geometry', 'highway', 'bearing', 'length']
new_network = GeoDataFrame(dic_final, crs=project_crs)

# create network
new_network.explore()
new_network.to_file(f'{data_folder}/simp.shp')

In [3]:
# starting point
new_network = gpd.read_file(f'{data_folder}/simp.shp')
new_network

,name,highway,bearing,group,is_simplif,geometry
0,11,path,106.60,path,0,"LINESTRING (857888.020 5626761.570, 857902.213..."
1,11,path,106.60,path,0,"LINESTRING (857638.976 5626560.639, 857618.671..."
2,18,path,40.85,path,0,"LINESTRING (860173.665 5627099.066, 860141.427..."
3,18,path,40.85,path,0,"LINESTRING (860173.665 5627099.066, 860196.674..."
4,18,path,164.90,path,0,"LINESTRING (860110.012 5627377.578, 860111.003..."
...,...,...,...,...,...,...
16264,"['Viale Enrico Millo', 'Viale Virgilio']",pedestrian,7.40,pedestrian,0,"LINESTRING (855537.353 5629215.626, 855536.028..."
16265,"['Viale Rombon', ""Viale Stretta D'Auzza""]",path,84.20,path,0,"LINESTRING (859391.734 5626943.304, 859404.759..."
16266,"['Viale San Gabriele', 'Viale Castelgomberto']",path,30.80,path,0,"LINESTRING (858361.706 5626215.887, 858343.194..."
16267,"['Viale Stefano Turr', 'Viale Virgilio']",pedestrian,27.10,pedestrian,0,"LINESTRING (855542.740 5629433.523, 855532.020..."


In [44]:
# region
# Classes to be employed during the execution of this code.
#Intersection
#Split in intersection
class Intersection:
    def __init__(self,network,number:int):
        """

        :param network:
        :param number: give a unique name to the files created during the process (this class will be use again in this code)
        """
        self.my_network = network
        self.inter_pnt_dic = {'geometry':[],'name':[]}
        self.lines_to_delete =[]
        self.num = number
    def delete_false_intersection(self):
        # First clean all the false node
        self.my_network = remove_false_nodes(self.my_network)
        # the previous function has changed the topolog so the length should be updated
        self.my_network['length'] =self.my_network.length
        self.my_network  =self.my_network.drop_duplicates(subset='length')
        self.my_network.reset_index(drop=True,inplace= True) # Changes has been made to the geometry so the index should be reset
        self.my_network = remove_false_nodes(self.my_network)
        self.my_network['length'] =self.my_network.length
        self.my_network  =self.my_network.drop_duplicates(subset='length')
        self.my_network.reset_index(drop=True,inplace= True)

    def intersection_network(self):

        # Create buffer around each element
        buffer_around_lines= self.my_network['geometry'].buffer(cap_style=3, distance=1, join_style=3)


        # s_join between buffer to lines
        s_join_0 =gpd.sjoin(left_df=GeoDataFrame(geometry=buffer_around_lines,crs=project_crs),right_df=self.my_network)

        # delete lines belong to the buffer
        s_join = s_join_0[s_join_0.index!=s_join_0['index_right']]


        # Find new intersections that are not at the beginning or end of the line
        for_time =len(s_join)
        with tqdm(total=for_time) as pbar:
            s_join.apply(lambda x: self.find_intersection_points(x,pbar), axis=1)
        if len(self.inter_pnt_dic)==0:
            return
        inter_pnt_gdf = GeoDataFrame(self.inter_pnt_dic,crs=project_crs)

        # Split string line by points
        segments = {'geometry':[],'org_id':[]}
        # Groupby points name (which is the line they should split)
        my_groups =  inter_pnt_gdf.groupby('name')
        for_time = len(my_groups)
        with  tqdm(total=for_time) as pbar:
            for group_pnts in my_groups :
                pbar.update(1)
                points  = group_pnts[1]
                points['is_split'] = True

                # get the line to split by comparing the name
                row = self.my_network.loc[group_pnts[0]]
                current = list(row.geometry.coords)
                points_line = [Point(x) for x in current]
                points_line_gdf = GeoDataFrame(geometry=points_line,crs=project_crs)
                points_line_gdf['is_split'] = False

                # append all the points together (line points and split points)
                line_all_pnts = points_line_gdf.append(points)

                # Find the distance of each point form the begining of the line on the line.
                line_all_pnts['dis_from_the_start'] = line_all_pnts['geometry'].apply(lambda x:row.geometry.project(x))
                line_all_pnts.sort_values('dis_from_the_start',inplace=True)

                # split the line
                seg =[]
                for point in line_all_pnts.iterrows():
                    prop = point[1]
                    seg.append(prop['geometry'])
                    if prop['is_split']:
                        segments['geometry'].append(LineString(seg))
                        segments['org_id'].append(row.name)
                        seg = [prop['geometry']]
                # if the split point is the last one, you don't need to create new segment
                if len(seg)>1:
                    segments['geometry'].append(LineString(seg))
                    segments['org_id'].append(row.name)
        network_split = GeoDataFrame(data=segments,crs=project_crs)
        cols_no_geometry = self.my_network.columns[:-1]
        network_split_final = network_split.set_index('org_id')
        network_split_final[cols_no_geometry] =self.my_network[cols_no_geometry]

        # remove old and redundant line from our network and update with new one
        network_split =self.my_network.drop(index=network_split_final.index.unique()).append(network_split_final).drop(index= self.lines_to_delete)
        network_split['length'] = network_split.length
        self.my_network = network_split


    def find_intersection_points(self,row,pbar):
        r"""
        find the intersection points between the two lines
        :param row:
        :return:
        """
        try:
            pbar.update(1)
            line_1 = self.my_network.loc[row.name]
            line_2 =  self.my_network.loc[row['index_right']]
            pnt = line_1.geometry.intersection(line_2.geometry)
            # If there are more than one intersection between two lines, one of the lines should be deleted.
            if isinstance(pnt,LineString):
                return
            if isinstance(pnt,MultiPoint):
                temp_line= line_1.name if line_1.length< line_2.length else line_2.name
                if temp_line not in self.lines_to_delete:
                    self.lines_to_delete.append(temp_line)
                return
            # If it is first or end continue OR if there is no intersection between the two lines
            if len(pnt.coords)==0 or pnt.coords[0]==line_1.geometry.coords[0] or pnt.coords[0]==line_1.geometry.coords[-1]:
                return
            self.inter_pnt_dic['geometry'].append(pnt)
            self.inter_pnt_dic['name'].append(row.name)
        except:
            print(f"{row.name},{row['index_right']}:{pnt}")


#Roundabout
class EnvEntity:
        def __init__(self,network):
            self.dead_end_fd = None
            self.pnt_dead_end = None
            self.pnt_dic = {}
            self.first_last_dic = {'geometry': [], 'line_name': [], 'position': []}
            self.network = network


        def __populate_pnt_dic(self,point: type, name_of_line: str):
            """
            Make "pnt_dic" contain a list of all the lines connected to each point.
            :param point:
            :param name_of_line:
            :return:
            """
            if not point in self.pnt_dic:
                self.pnt_dic[point] = []
            self.pnt_dic[point].append(name_of_line)

        def __send_pnts(self,temp_line: GeoSeries):
            """
            # Send the first and the last points to populate_pnt_dic
            :return:
            """
            my_geom = temp_line['geometry']
            self.__populate_pnt_dic(my_geom.coords[0], temp_line.name)
            self.__populate_pnt_dic(my_geom.coords[-1], temp_line.name)

        def get_deadend_gdf(self,delete_short:int =30)-> GeoDataFrame:
            self.network.apply(self.__send_pnts, axis=1)

            deadend_list = [item[1][0] for item in self.pnt_dic.items() if len(item[1]) == 1]
            pnt_dead_end_0 = [item for item in self.pnt_dic.items() if len(item[1]) == 1] # Retain all the line points with deadened
            self.pnt_dead_end = [Point(x[0]) for x in pnt_dead_end_0]
            # Create shp file of deadened_pnts
            geometry,line_name = 'geometry','line_name'
            pnt_dead_end_df = GeoDataFrame(data=pnt_dead_end_0)
            pnt_dead_end_df[geometry]= pnt_dead_end_df[0].apply(lambda x:Point(x))
            pnt_dead_end_df[line_name] = pnt_dead_end_df[1].apply(lambda x:x[0])
            pnt_dead_end_df.crs = project_crs
            self.dead_end_fd = pnt_dead_end_df

            if delete_short>0:
                # If it is necessary to eliminate dead-end short segments, it is  important to delete them from the network geodataframe.

                deadend_gdf =self.network.loc[deadend_list]
                self.network.drop(index=deadend_gdf[deadend_gdf.length<delete_short].index,inplace=True)
                return deadend_gdf[deadend_gdf.length>delete_short]
            return self.network.loc[deadend_list]

        def update_the_current_network(self,temp_network):
            r"""
            Update the current network in the new changes
            :param temp_network:
            :return:
            """
            new_network_temp = self.network.drop(index=temp_network.index)
            self.network = new_network_temp.append(temp_network)
            self.network['length'] = self.network.length
            self.network  = self.network[self.network['length']>1]
class Roundabout(EnvEntity):
    def __init__(self,network: GeoDataFrame):
       EnvEntity.__init__(self,network)
       self.pnt_dic ={}
       self.centroid =self.__from_roundabout_to_centroid()
       self.network.rename(columns={'name': 'str_name'}, inplace=True)
    def __from_roundabout_to_centroid(self):
        # Find the center of each roundabout
        # create polygon around each polygon and union
        round_about_buffer = round_about.to_crs(project_crs)['geometry'].buffer(cap_style=1, distance=10,
                                                                                join_style=1).unary_union
        dic_data = {'name': [], 'geometry': []}
        if round_about_buffer.type=='Polygon': # In case we have only one polygon
            dic_data['name'].append(0)
            dic_data['geometry'].append(round_about_buffer.centroid)
        else:
            for ii, xx in enumerate(round_about_buffer):
                dic_data['name'].append(ii)
                dic_data['geometry'].append(xx.centroid)
        centroid =GeoDataFrame(dic_data, crs=project_crs)
        return centroid
        # GeoDataFrame(dic_data,crs=project_crs).to_file(f'{path_round_about}/roundabout_union.shp')

    def __first_last_pnt_of_line(self,row: GeoSeries):
        r"""
        It get geometry of line and fill the first_last_dic with the first and last point and the name of the line
        :return:
        """
        geo = list(row['geometry'].coords)
        self.first_last_dic['geometry'].extend([Point(geo[0]), Point(geo[-1])])
        self.first_last_dic['line_name'].extend([row.name] * 2)
        self.first_last_dic['position'].extend([0, -1])
    def deadend(self):
        r"""
        remove not connected line shorter than 100 meters and then return deadend_list lines and their endpoints (as another file)
        :return:
        """
        # Find the first and last points

        # Get deadend_gdf
        deadend_gdf = self.get_deadend_gdf()

        # Create gdf of line points with the reference to the line they belong
        deadend_gdf.apply(self.__first_last_pnt_of_line, axis=1)
        first_last_gdf = GeoDataFrame(self.first_last_dic, crs=project_crs)


        return deadend_gdf, first_last_gdf
    def __update_geometry(self,cur,s_join):
        r"""
        :return:
        """
        if cur['highway'] == 'footway':
            # Don't snap footway to roundabout
            return cur['geometry']
        # Get only the points that are deadened
        points_lines = [item for item in s_join[s_join['line_name'] == cur.name].iterrows()if item[1]['geometry'] in self.pnt_dead_end]
        if len(points_lines) == 0:
            # No roundabout nearby
            return cur['geometry']
        # get the line geometry to change the first and/ or last point
        geo_cur = list(cur['geometry'].coords)

        # iterate over the deadened points  near roundabout
        for ind in range(len(points_lines)):
            points_line = points_lines[ind]
            geo_cur[points_line[1]['position']] = self.centroid.loc[points_line[1]['index_right']]['geometry'].coords[
                0]
        return LineString(geo_cur)
    def my_spatial_join(self,deadend_lines, deadend_pnts,line_name):
        # Spatial join between roundabout centroid to nearby dead end lines
        # centroid = gpd.read_file(f'{path_round_about}/centroid.shp')
        s_join = gpd.sjoin_nearest(left_df=deadend_pnts, right_df=self.centroid, how='left', max_distance=100,
                                   distance_col='dist').dropna(subset='dist')

        # Deadened lines from both lines should be removed
        lines_to_delete_test = s_join['line_name'].unique() # all the Deadened lines close to roundabout

        # All deadened lines from both lines
        deads_both_side = self.dead_end_fd['line_name'].value_counts()
        deads_both_side =deads_both_side[deads_both_side==2]

        # Remove this lines from the database
        lines_to_delete=deads_both_side[deads_both_side.index.isin(lines_to_delete_test)]

        self.network = self.network[~((self.network[line_name].isin(lines_to_delete.index)) & (self.network.length<300))]
        deadend_lines = deadend_lines[~((deadend_lines[line_name].isin(lines_to_delete.index)) & (deadend_lines.length<300))]
        # Update the geometry so the roundabout will be part of the line geometry
        change_geo = deadend_lines.copy()

        change_geo['geometry'] = change_geo.apply(lambda x:self.__update_geometry(x,s_join), axis=1)

        return change_geo
# endregion

In [45]:

num=0
new_gpd = new_network.copy()
obj_intersection = Intersection(new_gpd,num)
obj_intersection.delete_false_intersection()

obj_intersection.my_network.to_file(f'{data_folder}/top_0_0.shp')

In [38]:
# The code intends to find errors in intersection_network
testmy_network  =obj_intersection.my_network.drop_duplicates(subset='length')
testmy_network.to_file(f'{data_folder}/remove_duplicate.shp')

In [39]:
# Create buffer around each element
buffer_around_lines= testmy_network['geometry'].buffer(cap_style=3, distance=1, join_style=3)
buffer_around_lines.to_file(f'{data_folder}/buffer_around_lines.shp')

In [32]:
buffer_around_lines

0        POLYGON ((857902.263 5626761.860, 857902.510 5...
1        POLYGON ((857618.570 5626561.707, 857618.170 5...
2        POLYGON ((860196.509 5627103.911, 860196.639 5...
3        POLYGON ((864243.827 5632242.688, 864244.011 5...
4        POLYGON ((854530.101 5631880.203, 854528.998 5...
                               ...                        
10222    POLYGON ((864626.246 5634117.596, 864626.186 5...
10223    POLYGON ((864653.482 5634020.951, 864655.990 5...
10224    POLYGON ((864656.280 5633635.234, 864656.282 5...
10225    POLYGON ((864894.132 5633455.372, 864869.998 5...
10226    POLYGON ((865099.156 5633833.690, 865106.373 5...
Length: 10226, dtype: geometry

In [40]:
data_2  =GeoDataFrame(geometry=buffer_around_lines,crs=project_crs)
data_2 .to_file(f'{data_folder}/buffer_around_lines2.shp')

In [24]:
testmy_network

,name,highway,bearing,group,is_simplif,geometry,length
0,11,path,106.60,path,0.0,"LINESTRING (857888.020 5626761.570, 857902.213...",563.914540
1,11,path,106.60,path,0.0,"LINESTRING (857638.976 5626560.639, 857618.671...",309.186141
2,18,path,40.85,path,0.0,"LINESTRING (860173.665 5627099.066, 860196.674...",173.627941
3,600,path,148.80,path,0.0,"LINESTRING (864250.196 5632236.081, 864244.618...",950.075672
4,Arsenale n°2010,footway,113.90,footway,0.0,"LINESTRING (854586.406 5631852.497, 854529.644...",67.337958
...,...,...,...,...,...,...,...
10222,NaN,NaN,NaN,NaN,NaN,"LINESTRING (864637.153 5634144.738, 864625.331...",146.559029
10223,NaN,NaN,NaN,NaN,NaN,"LINESTRING (864648.085 5634018.856, 864654.174...",135.606386
10224,NaN,NaN,NaN,NaN,NaN,"LINESTRING (864624.775 5633635.354, 864656.245...",525.076466
10225,NaN,NaN,NaN,NaN,NaN,"LINESTRING (864901.147 5633454.840, 864894.324...",821.115766


In [31]:
data_2

,geometry
0,"POLYGON ((857902.263 5626761.860, 857902.510 5..."
1,"POLYGON ((857618.570 5626561.707, 857618.170 5..."
2,"POLYGON ((860196.509 5627103.911, 860196.639 5..."
3,"POLYGON ((864243.827 5632242.688, 864244.011 5..."
4,"POLYGON ((854530.101 5631880.203, 854528.998 5..."
...,...
10222,"POLYGON ((864626.246 5634117.596, 864626.186 5..."
10223,"POLYGON ((864653.482 5634020.951, 864655.990 5..."
10224,"POLYGON ((864656.280 5633635.234, 864656.282 5..."
10225,"POLYGON ((864894.132 5633455.372, 864869.998 5..."


In [41]:

# s_join between buffer to lines
s_join_0 =gpd.sjoin(left_df=data_2,right_df=testmy_network)
s_join.to_file(f'{data_folder}/s_join_0.shp')

In [ ]:
# delete lines belong to the buffer
s_join = s_join_0[s_join_0.index!=s_join_0['index_right']]
s_join.to_file(f'{data_folder}/s_join.shp')

In [ ]:

obj_intersection.intersection_network()
obj_intersection.my_network.to_file(f'{data_folder}/top_0.shp')

In [10]:

line_name ='line_name'
if is_junction:

    exist_data= obj_intersection.my_network.reset_index().reset_index(names=line_name)
    my_roundabout=Roundabout(exist_data)
    deadend_lines, deadend_pnts = my_roundabout.deadend()

    # update the current network
    change_geo = my_roundabout.my_spatial_join(deadend_lines, deadend_pnts,line_name)
    my_roundabout.update_the_current_network(change_geo)

    my_roundabout.network.drop_duplicates(subset=line_name,inplace=True)
    # Improve roundabout
    # First buffer around centroid
    centr_name= 'centr_name'
    buffer_around_centroid= my_roundabout.centroid['geometry'].buffer(cap_style=1, distance=30)

    # s_join between buffer to lines (reset index to retain the original centroid name which can apper more than one in the results). always stay with data you need and with understandable name
    roundabout_with_lines =gpd.sjoin(left_df=GeoDataFrame(geometry=buffer_around_centroid,crs=project_crs).reset_index(),right_df=my_roundabout.network[['geometry',line_name]]).drop_duplicates(subset=['index',line_name]).rename(columns={"index":centr_name})[['geometry',line_name,centr_name]]

    # To facilitate the searching process
    my_roundabout.network.set_index(line_name,inplace=True)
    # To facilitate easy access to point centroid geometry data, it is advisable to store the information in an object that provides efficient retrieval.
    pnt_centroid_temp = my_roundabout.centroid['geometry']
    #  Group the data by centroid
    for center_line in roundabout_with_lines.groupby(centr_name):
        #  Iterate over each group after performing a groupby() operation
        for center in center_line[1].itertuples():
            # Find the line that connects to the current centroid and obtain its vertices
            line_to_test = my_roundabout.network.loc[center[2]]
            vertices_line = list(line_to_test['geometry'].coords)
            pnt_test = [vertices_line[0],vertices_line[-1]]
            # To determine if the current line is already connected to the current centroid,.
            is_connected = my_roundabout.centroid[my_roundabout.centroid['geometry'].isin([Point(pnt_test[0]),Point(pnt_test[-1])])]
            if len(is_connected)>0 and center[3] in is_connected['name']:
                continue

            if len(vertices_line)==2:
                vertices_line.insert(1, pnt_centroid_temp[center[3]])
            else:
                my_list = [pnt_centroid_temp[center[3]].distance(Point(temp)) for temp in vertices_line]
                # Find the minimum index
                min_index = min(range(len(my_list)), key=my_list.__getitem__)
                if min_index ==0:
                    vertices_line.insert(0,pnt_centroid_temp[center[3]])
                elif min_index == len(my_list)-1:
                    vertices_line.append(pnt_centroid_temp[center[3]])
                else:
                    vertices_line[min_index] = pnt_centroid_temp[center[3]]
            new_geo = LineString(vertices_line)
            my_roundabout.network.at[center[2],'geometry'] = new_geo
# Extend
    new_network2 = my_roundabout.network.reset_index()
else:
    new_network2=  obj_intersection.my_network.reset_index()
new_network2.to_file(f'{data_folder}/ra_network.shp')

In [6]:

extend_lines_f= extend_lines(new_network2,100)
extend_lines_f['length'] = extend_lines_f.length
extend_lines_f.to_file(f'{data_folder}/extend_network.shp')

In [ ]:


obj_intersection_1 = Intersection(extend_lines_f.copy(),1)
obj_intersection_1.delete_false_intersection()
obj_intersection_1.intersection_network()
# Clear short segments
final2 = EnvEntity(obj_intersection_1.my_network.reset_index())
final2.update_the_current_network(final2.get_deadend_gdf(delete_short=30))
final2.network.to_file(f'{data_folder}/final.shp')